# Song Recommendation

This code does the following:-
1. Parse the input text le and create the user-song matrix M.
2. Preprocess the data by binning the play counts into b bins.
3. Preprocess the data to avoid the cold start issue, and getting the final shape as (5693; 10966)
4. set-aside 200 randomly selected non-zero data-points from the matrix as a test set.
5. Performing AO and calculating RMSE.

In [3]:
import re
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spl
import sys
import matplotlib.pyplot as plt
import random
import time
from scipy import sparse
from sklearn.linear_model import Ridge


file_path = "train_triplets.txt"
# The sparse matrix function
def create_sparse_matrix(users, songs, play_count, row=True):
    if row:
        return sp.csr_matrix((play_count, (users, songs)), shape=(len(users), len(songs)))
    else:
        return sp.csc_matrix((play_count, (users, songs)), shape=(len(users), len(songs)))

# The parse triplets function
def parse_triplets(file_path, max_rows, whole_dataset, b, use_vectors, use_dikt):
    # Vectors

    users = []
    songs = []
    play_count = []

    mapping_users = {}
    mapping_songs = {}
    
    # Code to extract users, songs and playcount from the train_triplets.txt 

    triplets = open(file_path, 'r')
    
    line_regex = re.compile("^([\w\d]+)\t([\w\d]+)\t([\w]+)$")

    current_row = 0

    highest_playcount = 0

    for triplet in triplets:
        triplet_match = line_regex.match(triplet)

        if (use_vectors):
            if triplet_match.group(1) not in mapping_users:
                mapping_users[triplet_match.group(1)] = len(mapping_users)
    
            if triplet_match.group(2) not in mapping_songs:
                mapping_songs[triplet_match.group(2)] = len(mapping_songs)

            users.append(int(mapping_users[triplet_match.group(1)]))
            songs.append(int(mapping_songs[triplet_match.group(2)]))
            play_count.append(int(triplet_match.group(3)))

        if int(triplet_match.group(3)) > highest_playcount:
            highest_playcount = int(triplet_match.group(3))

        current_row += 1
        if current_row >= max_rows and not whole_dataset:
            break

    triplets.close()

    print("Highest playcount: ", highest_playcount)

    return (users, songs, play_count)

    
def binning(play_count, bin_size):
    bin_array = [2**i for i in range(bin_size)]
    print("Bin Array : ", bin_array)
    binned_counts = np.digitize(play_count, bin_array, right = False)
    print("Binned Counts : ", binned_counts)
    return binned_counts
        
    
def cold_start(user_song_matrix, min_threshold):
    #We need to remove songs & users with less or equal to min_threshold 
    #print(user_song_matrix)
    user_song_matrix = user_song_matrix > 0
    user_song_matrix = user_song_matrix.astype(np.int)
    #print(user_song_matrix.toarray())
        
    songs_per_user = np.ravel(np.sum(user_song_matrix, axis=1)) #sum of row
    users_to_delete = np.where(songs_per_user <= min_threshold)[0]
    
    ##set the users i.e rows to 0
    for i in users_to_delete:
            user_song_matrix.data[user_song_matrix.indptr[i]:user_song_matrix.indptr[i+1]]=0

    user_song_matrix.eliminate_zeros()
    mask = np.concatenate(([True], user_song_matrix.indptr[1:] != user_song_matrix.indptr[:-1]))
    user_song_matrix = sp.csr_matrix((user_song_matrix.data, user_song_matrix.indices, user_song_matrix.indptr[mask]))

    ##set the songs i.e columns to 0
    user_song_matrix = sp.csc_matrix(user_song_matrix)
    
    users_per_song = np.ravel(np.sum(user_song_matrix, axis=0)) #sum of column
    songs_to_delete = np.where(users_per_song <= min_threshold)[0]

    for i in songs_to_delete:
            user_song_matrix.data[user_song_matrix.indptr[i]:user_song_matrix.indptr[i+1]]=0

    user_song_matrix.eliminate_zeros()
    mask = np.concatenate(([True], user_song_matrix.indptr[1:] != user_song_matrix.indptr[:-1]))
    user_song_matrix = sp.csc_matrix((user_song_matrix.data, user_song_matrix.indices, user_song_matrix.indptr[mask]))	
    
    return user_song_matrix.tocsr()

def singular_value_decomp(sparse_matrix):
    sparse_matrix = sparse_matrix.asfptype()
    #print("Matrix shape : ", sparse_matrix.shape)
    # print("Sparse Matrix : ", sparse_matrix)
    
    U, sigma, Vt = spl.svds(sparse_matrix, k=30)
    #print("U shape : ", U.shape, " sigma shape : ", sigma.shape, " V shape : ", Vt.shape)
    
    sigma = np.diag(sigma)
    #print("Sigma shape after converting to diagonal matrix : ", sigma.shape)
    Q = U.dot(sigma)
    #print("Q shape : ", Q.shape)
    #print("Pt shape : ", Vt.shape)
    return Q, Vt

def Alternating_optimization(Q,Pt, user_song_matrix):

    dot_sum = 0
    Qt = np.transpose(Q)
    user_song_matrix = sp.csr_matrix(user_song_matrix)

    diff = 99999
    pre_diff = float('inf')
    while(diff != 0):
        

        A = minimize(Q, user_song_matrix.todense())
        B = minimize(A, user_song_matrix.todense().transpose())
        #print("Shapes : ")
        #print("A shape : ", A.shape)
        #print("B shape : ", B.shape)
        S = user_song_matrix - np.dot(B, A.T)
        #S.data = np.power(S.data, 2)
        S = np.power(S, 2)
        diff = np.sum(np.sum(S,axis=1),axis=0)
        
        real_diff = np.absolute(pre_diff - diff)

        if real_diff < 0.5:
            break
        else:
            pre_diff = diff
        #print("Difference is : ", real_diff)
        
        Q = B
       
    Mdiff = np.dot(B, A.T)
    #diff_final = np.linalg.norm(user_song_matrix - Mdiff)
    
    #print(diff_final)
    return Mdiff

def minimize(data, target):
    clf = Ridge(alpha=10.0, fit_intercept=False)
    clf.fit(data, target)
    return clf.coef_

#
#   This functions picks a random set of size: number_of_random_elements
#   
#   Returns: M_s the modified M
#            random_set containing all random pciked elements on the form [(row,col,data)]
#
def pick_random_test_set(M, number_of_random_elements):
    random.seed(time.time())
    
    M_s = sp.coo_matrix(M)
    interval = len(M_s.data)

    random_picked_values = []
    already_picked_values = []
    
    for i in range(number_of_random_elements):
        random_index = random.randint(0, interval-1)

        while random_index in already_picked_values:
            random_index = random.randint(0, interval-1)
        already_picked_values.append(random_index)
        
        random_picked_values.append((M_s.row[random_index], 
                                    M_s.col[random_index], 
                                    M_s.data[random_index]))
        M_s.data[random_index] = 0
        #print(random_picked_values[len(random_picked_values)-1])

    M_s.eliminate_zeros()
    M_s = sp.csr_matrix(M_s)
    return M_s, random_picked_values

def rmse(Mdiff, test_set):
    
    y_sum = 0
    for test in test_set:
        y_sum += (test[2] - Mdiff[test[0]][test[1]])**2

    rmse = np.sqrt(y_sum/len(test_set))

    print("RMSE: ", rmse)
    
if len(sys.argv) < 2:
    print("To few arguments...")
    sys.exit(1)

min_threshold = 5
bin_size = 10 ## This is configurable by user
size_of_test_set = 200

print("Parsing the triplets...")
users, songs, play_count = parse_triplets(file_path, 300000, False, 10, True, False)

# Here the logic for the binnig should be placed

binned_play_counts = binning(play_count, bin_size)

# Either pass this binned_play_counts to the following functions that require play_count
# as a parameter, or change the return variable "binned_play_counts" to "play_count"
# to override its value

# Easiest way to create a sparse matrix is done by using the vectors
resulting_sparse_matrix = create_sparse_matrix(users, songs, binned_play_counts, row=True)
print("Done parsing the triplets -> Sparse matrix")

shape_orig = resulting_sparse_matrix.shape
while True:
    shape_before = resulting_sparse_matrix.shape
    resulting_sparse_matrix = cold_start(resulting_sparse_matrix, min_threshold)
    shape_after = resulting_sparse_matrix.shape
    print("Intermediate shape: ", shape_after)
    if shape_before[0] == shape_after[0] and shape_before[1] == shape_after[1]:
        break
shape_final = resulting_sparse_matrix.shape

print("Picking random test set... ")
resulting_sparse_matrix, test_set = pick_random_test_set(resulting_sparse_matrix, size_of_test_set)

#print("Got random test set: ", test_set)

# Initial P & Q values obtained using SVD
print("Calculating SVD...")
Q, Pt = singular_value_decomp(resulting_sparse_matrix)
# Perform AO using P,Q
print("Doing Alternating optimization...")
Mdiff = Alternating_optimization(Q,Pt, resulting_sparse_matrix)
print("Orig shape: ", shape_orig, " Final shape: ", shape_after)

print("Calculating RMSE...")
rmse(Mdiff, test_set)

sys.stdout.flush()


Parsing the triplets...
Highest playcount:  1890
Bin Array :  [1, 2, 4, 8, 16, 32, 64, 128, 256, 512]
Binned Counts :  [1 1 2 ..., 1 1 1]
Done parsing the triplets -> Sparse matrix
Intermediate shape:  (6270, 11212)
Intermediate shape:  (5716, 10981)
Intermediate shape:  (5695, 10966)
Intermediate shape:  (5693, 10966)
Intermediate shape:  (5693, 10966)
Picking random test set: 
Orig shape:  (300000, 300000)  Final shape:  (5693, 10966)
RMSE:  0.966275866005
